In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
plt.style.use('ggplot')

from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import homogeneity_completeness_v_measure
from scipy.spatial import distance


from scipy.linalg import eigh
import matplotlib.pyplot as plt

import os
import pytest
from dadapy import Data
from dadapy._utils import utils as ut
from scipy.optimize import minimize
from scipy.linalg import eigh, qr, solve, svd
from scipy.sparse import csr_matrix, eye
from scipy.sparse.linalg import eigsh
from scipy.stats import ks_2samp
import sklearn
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance
from sklearn.manifold import SpectralEmbedding

rng = np.random.default_rng()

import warnings
warnings.filterwarnings('ignore')

In [2]:
import random 
np.random.seed(0)
random.seed(0)


n_osservazioni = 1700
n_variabili = 17
R, r = 2, 1  # Parametri per il toro
a, b = 0.1, 0.1  # Parametri per la spirale 3D
r_sfera = 1  # Raggio della sfera


# Generazione del toro
theta = np.linspace(0, 2*np.pi, n_osservazioni)
phi = np.linspace(0, 2*np.pi, n_osservazioni)
x_toro = (R + r * np.cos(theta)) * np.cos(phi)
y_toro = (R + r * np.cos(theta)) * np.sin(phi)
z_toro = r * np.sin(theta)
toro = np.column_stack((x_toro, y_toro, z_toro))
toro = np.hstack([toro]+[np.random.normal(0, 0.05, (n_osservazioni, 3))] + [np.random.normal(0, 0.05, (n_osservazioni, n_variabili - 3))])

# Generazione della spirale 3D
t = np.linspace(0, 20*np.pi, n_osservazioni)
x_spirale = a * t * np.cos(t)
y_spirale = a * t * np.sin(t)
z_spirale = b * t
spirale_3d = np.column_stack((x_spirale, y_spirale, z_spirale))
spirale_3d = np.hstack([spirale_3d]+[np.random.normal(0, 0.05, (n_osservazioni, 3))] + [np.random.normal(0, 0.05, (n_osservazioni, n_variabili - 3))])

np.random.seed(0)
# Generazione della sfera
phi = np.random.uniform(0, np.pi, n_osservazioni)
theta = np.random.uniform(0, 2*np.pi, n_osservazioni)
x_sfera = r_sfera * np.sin(phi) * np.cos(theta)
y_sfera = r_sfera * np.sin(phi) * np.sin(theta)
z_sfera = r_sfera * np.cos(phi)
sfera = np.column_stack((x_sfera, y_sfera, z_sfera))
sfera = np.hstack([sfera]+[np.random.normal(0, 0.05, (n_osservazioni, 3))] + [np.random.normal(0, 0.05, (n_osservazioni, n_variabili - 3))])

# Unione dei manifold in un unico DataFrame
dataframes_complessi = [pd.DataFrame(toro), pd.DataFrame(spirale_3d), pd.DataFrame(sfera)]
X = pd.concat(dataframes_complessi, ignore_index=True).values.astype(float)
X.shape

y = np.repeat(np.arange(0, 3), n_osservazioni)

In [3]:
class AdaptiveSpectralClustering:
    def __init__(self, X, n_iter=10):
        """
        Initialize the clustering class with data matrix X.
        
        Args:
            X: Input data matrix of shape (n_samples, n_features)
            n_iter: Number of iterations for ID estimation
        """
        self.X = X
        self.n_iter = n_iter
        self.ids = None
        self.kstars = None
        self.similarity_matrix = None
        self.n_components = None
        self.neighs_ind = None
        
    def return_ids_kstar_binomial(self, initial_id=None, Dthr=6.67, r='opt', verbose=True):
        """
        Return the id estimates of the binomial algorithm coupled with the kstar estimation of the scale.
        
        Args:
            initial_id: Initial estimate of the id (default uses 2NN)
            Dthr: Threshold value for the kstar test
            r: Parameter of binomial estimator, 0 < r < 1
            verbose: Whether to print progress
        """
        data = Data(self.X)
        
        if initial_id is None:
            data.compute_id_2NN(algorithm='base')
        else:
            data.compute_distances()
            data.set_id(initial_id)

        ids = np.zeros(self.n_iter)
        kstars = np.zeros((self.n_iter, data.N), dtype=int)

        for i in range(self.n_iter):
            data.compute_kstar(Dthr)
            if verbose:
                print("iteration ", i)
                print("id ", data.intrinsic_dim)

            r_eff = min(0.95, 0.2032**(1./data.intrinsic_dim)) if r == 'opt' else r
            rk = np.array([dd[data.kstar[j]] for j, dd in enumerate(data.distances)])
            rn = rk * r_eff
            n = np.sum([dd < rn[j] for j, dd in enumerate(data.distances)], axis=1)
            
            id = np.log((n.mean() - 1) / (data.kstar.mean() - 1)) / np.log(r_eff)
            data.set_id(id)

            ids[i] = id
            kstars[i] = data.kstar

        self.ids = ids
        self.kstars = kstars[(self.n_iter - 1), :]
        return self.ids, self.kstars

    def find_Kstar_neighs(self):
        """Find K* nearest neighbors for each point."""
        nn = NearestNeighbors(n_jobs=-1)
        nn.fit(self.X)

        neighs_ind = []
        for i, obs in enumerate(self.X):
            distance, ind = nn.kneighbors([obs], n_neighbors=int(self.kstars[i]) + 1)
            k_neighs = ind[0][1:]
            neighs_ind.append(k_neighs.tolist())
            
        self.neighs_ind = neighs_ind
        return self.neighs_ind

    def find_components(self):
        """Find number of components based on intrinsic dimension."""
        self.n_components = int(np.round(self.ids[self.n_iter-1]))
        return self.n_components

    def create_similarity_matrix(self, use_distances=False):
        """
        Create similarity matrix based on K* neighbors.
        
        Args:
            use_distances: If True, use euclidean distances instead of binary weights
        """
        similarity_matrix = np.zeros((self.X.shape[0], self.X.shape[0]))
        
        if use_distances:
            for i in range(similarity_matrix.shape[0]):   
                distances = [distance.euclidean(self.X[i, :], self.X[j, :]) 
                           for j in self.neighs_ind[i]]
                similarity_matrix[i, self.neighs_ind[i]] = distances
        else:
            for i in range(similarity_matrix.shape[0]):
                similarity_matrix[i, self.neighs_ind[i]] = 1.0
        
        self.similarity_matrix = 0.5 * (similarity_matrix + similarity_matrix.T)
        return self.similarity_matrix

    def return_clustering(self, n_clusters, use_n_components=True):
        """
        Perform spectral clustering using Laplacian eigenvectors.
        
        Args:
            n_clusters: Number of clusters to create
            use_n_components: If True, use estimated intrinsic dimension for embedding
        """
        if use_n_components:
            k = self.n_components
        else:
            k = n_clusters
            
        degree_matrix = np.diag(self.similarity_matrix.sum(axis=1))
        laplacian_matrix = degree_matrix - self.similarity_matrix
        eigenvalues, eigenvectors = eigh(laplacian_matrix)
        k_eigenvectors = eigenvectors[:, :k]
        
        kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
        kmeans.fit(k_eigenvectors)
        return kmeans.labels_

    def return_clusters(self, n_clusters):
        """
        Alternative clustering method using SpectralEmbedding.
        
        Args:
            n_clusters: Number of clusters to create
        """
        se = SpectralEmbedding(n_components=3, random_state=0, affinity='precomputed')
        embs = se.fit_transform(self.similarity_matrix)
        
        km = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
        km.fit(embs)
        return km.labels_

    def fit(self, n_clusters, use_distances=False, clustering_method='spectral'):
        """
        Complete pipeline for clustering.
        
        Args:
            n_clusters: Number of clusters to create
            use_distances: Whether to use distances in similarity matrix
            clustering_method: 'spectral' or 'embedding' for different clustering approaches
        
        Returns:
            cluster_labels: Array of cluster assignments
        """
        #Compute IDs and k*
        self.return_ids_kstar_binomial()
        
        #Find neighbors and create similarity matrix
        self.find_Kstar_neighs()
        self.create_similarity_matrix(use_distances=use_distances)
        
        #Determine number of components
        self.find_components()
        
        if clustering_method == 'spectral':
            return self.return_clustering(n_clusters=n_clusters)
        else:
            return self.return_clusters(n_clusters=n_clusters)



asc = AdaptiveSpectralClustering(X, n_iter=10)
labels = asc.fit(n_clusters=3)

iteration  0
id  [10.5]
iteration  1
id  [5.41]
iteration  2
id  [4.56]
iteration  3
id  [4.05]
iteration  4
id  [3.69]
iteration  5
id  [3.46]
iteration  6
id  [3.37]
iteration  7
id  [3.32]
iteration  8
id  [3.31]
iteration  9
id  [3.3]


In [4]:
print(adjusted_rand_score(y, labels))
print(homogeneity_completeness_v_measure(y, labels))

0.688522488538607
(0.7059516899737565, 0.7235396249489138, 0.7146374597222298)


In [6]:
# Spectral Clustering Sklearn default implementation based on Nearest Neighbors approach
from sklearn.cluster import SpectralClustering

sc = SpectralClustering(random_state=0, n_clusters = 3, affinity='nearest_neighbors')
preds_sc = sc.fit_predict(X)

print(adjusted_rand_score(y, preds_sc))
print(homogeneity_completeness_v_measure(y, preds_sc))

0.2626520735402502
(0.3671484155070117, 0.5332044852557576, 0.43486322249205406)
